In [1]:
# %%
# Importation des libraries/packages de base. 

import pandas as pd
import os
import numpy as np
import dask.dataframe as dd 
import warnings 
warnings.filterwarnings("ignore")

# %%
# Fichier avec les variables d'intéret : dernière version : 27/02 

df_cct = pd.read_parquet("/home/onyxia/work/df_cct_final.parquet1")

# %%
# Pour réaliser un test de Student : 
import scipy.stats as stats

# %%
# But : Comparaison de deux distribution : Ex : 
X1 = [1,1,1,2,1] 

X2 = [4,2,3,1,1]

y = stats.ttest_ind(X1,X2) ; print(y)

# %%
# Affichage de la p-value : 

print(y[1])

# %%
# Pour notre dataset : 
# Idée : Comparer les distributions des frauds et des non-frauds : 
# Le faire pour les montants et pour les fréquences de transactions. 

# %%
# Récupérer les frauds et les non-frauds : 
df_cct_fraud = df_cct[df_cct["Is Fraud?"]==1]
df_cct_non_fraud = df_cct[df_cct["Is Fraud?"]==0]

# %%
# Amount : 
Amount_fraud = df_cct_fraud["Amount"].to_numpy()
Amount_non_fraud = df_cct_non_fraud["Amount"].to_numpy()

# %%
# Vérifier les prérequis : Normalités des échnatillons : 

from scipy.stats import shapiro

x1, pval1 = shapiro(Amount_fraud)

x2, pval2 = shapiro(Amount_non_fraud)

print(pval1)

print(pval2)

# On est pas sur une distribution normale... Pas de sens de faire un test de student... 

# %%
# Test de student : 
y = stats.ttest_ind(Amount_fraud,Amount_non_fraud)
y

Ttest_indResult(statistic=-1.6222142113076257, pvalue=0.14341462050621775)
0.14341462050621775
0.0
0.0


Ttest_indResult(statistic=136.74731095326234, pvalue=0.0)

In [3]:
# Solution : test non-paramétrique : The Wilcoxon signed-rank test
Amount_non_fraud_1 = pd.DataFrame(Amount_non_fraud).sample(len(Amount_fraud)).to_numpy()
from scipy.stats import wilcoxon
res = wilcoxon(x = Amount_fraud - Amount_non_fraud_1, method = 'approx', zero_method='wilcox')
res.statistic, res.pvalue


## NON : on doit avoir de la dépendance entre les deux jeux de données : pas le cas entre les frauds et 
# non-frauds : ne s'applique pas. 

(array([79137710.5, 89778118. , 41609004.5, ..., 36521821.5, 67655508.5,
        16238203. ]),
 array([0., 0., 0., ..., 0., 0., 0.]))

In [2]:
# Le test de Wilcoxon-Mann-Whitney 
# (ou test U de Mann-Whitney ou encore test de la somme des rangs de Wilcoxon) : 

from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(Amount_fraud, Amount_non_fraud, method='asymptotic')
p


0.0

In [ ]:
# p-value = 0... On rejette toujours l'hypothèse nulle, les distributions sont très différentes... 